1. 安装maven, 参考：https://blog.csdn.net/qq_29695701/article/details/90705181

2. 安装pyspark==2.4.0

3. 按照github: https://github.com/titicaca/spark-iforest
里提示的两个步骤进行操作



In [1]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jdk/jdk1.8.0_191'
os.environ['SPARK_HOME'] = '/usr/lib/spark/spark-2.4.4-bin-hadoop2.7'


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
import tempfile


spark = SparkSession \
        .builder.master("local[*]") \
        .appName("IForestExample") \
        .getOrCreate()

data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([7.0, 9.0]),),
        (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]

# NOTE: features need to be dense vectors for the model input
df = spark.createDataFrame(data, ["features"])



/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
type(df)

pyspark.sql.dataframe.DataFrame

In [4]:
df.count()

4

In [5]:
df.show()

+---------+
| features|
+---------+
|[0.0,0.0]|
|[7.0,9.0]|
|[9.0,8.0]|
|[8.0,9.0]|
+---------+



In [6]:
from pyspark_iforest.ml.iforest import *

# Init an IForest Object
iforest = IForest(contamination=0.3, maxDepth=2)

# Fit on a given data frame
model = iforest.fit(df)

# Check if the model has summary or not, the newly trained model has the summary info
model.hasSummary



True

In [7]:

# Show model summary
summary = model.summary

# Show the number of anomalies
summary.numAnomalies

# Predict for a new data frame based on the fitted model
transformed = model.transform(df)

# Collect spark data frame into local df
rows = transformed.collect()

temp_path = tempfile.mkdtemp()
iforest_path = temp_path + "/iforest"

# Save the iforest estimator into the path
iforest.save(iforest_path)

# Load iforest estimator from a path
loaded_iforest = IForest.load(iforest_path)

model_path = temp_path + "/iforest_model"

# Save the fitted model into the model path
model.save(model_path)

# Load a fitted model from a model path
loaded_model = IForestModel.load(model_path)

# The loaded model has no summary info
loaded_model.hasSummary

# Use the loaded model to predict a new data frame
loaded_model.transform(df).show()

+---------+-------------------+----------+
| features|       anomalyScore|prediction|
+---------+-------------------+----------+
|[0.0,0.0]| 0.6575333612057278|       1.0|
|[7.0,9.0]| 0.3598940106267502|       0.0|
|[9.0,8.0]|0.44716436916437347|       0.0|
|[8.0,9.0]| 0.3351858475368414|       0.0|
+---------+-------------------+----------+

